**Entity Embeddings of Categorical Variables**
> https://arxiv.org/abs/1604.06737

**Generating Live Soccer-Math commentary from Play Data**
> https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwicl4Xi5J7wAhV7GDQIHW5lAOoQFjACegQIAxAD&url=https%3A%2F%2Fojs.aaai.org%2Findex.php%2FAAAI%2Farticle%2Fview%2F4691%2F4569&usg=AOvVaw0f2DIG3tjdsCeu_CJQ_2qW


In [3]:
!pip install fastai
!pip install fastcore
!pip install nbdev

     |████████████████████████████████| 193 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 12.8 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 776.8 MB 13 kB/s  eta 0:00:012     |███████████████                 | 365.7 MB 3.1 MB/s eta 0:02:14     |███████████████▊                | 382.6 MB 2.6 MB/s eta 0:02:31     |████████████████████▎           | 493.4 MB 2.6 MB/s eta 0:01:49     |███████████████████████         | 556.9 MB 2.7 MB/s eta 0:01:23     |█████████████████████████▎      | 613.6 MB 3.0 MB/s eta 0:00:54     |██████████████████████████████▍ | 738.1 MB 3.1 MB/s eta 0:00:13
     |████████████████████████████████| 10.4 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 923 kB/s eta 0:00:01
  Using cached tqdm-4.60.0-py2.py3-none-any.whl (75 kB)
     |████████████████████████████████| 184 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 9.8 MB 963 kB/s eta 0:00:01
  Using cached catalogue-1.0.0-py2.py3-none-any.wh

In [1]:
import pandas as pd
import numpy as np
import torch
from fastai.tabular.all import *
from fastcore.utils import *

/home/ROHITGANGURDE/anaconda3/envs/a7/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


# Load Data

In [2]:
df = pd.read_pickle('event_data.pickle')
df.head()

,time,event,player_sub,main_player,commentary
0,89',Yellow Card,,K. Phillips,"A hasty challenge from Kalvin Phillips now and his booking is more costly than usual; it is his fifth, earning him a one-match ban."
1,87',Substitution,H. Kane,Carlos Vinícius,"Harry Kane - who became a father this week - makes way for Tottenham's final change, with Carlos Vinicius on his place."
2,85',Yellow Card,,P. Højbjerg,"Hojbjerg picks up a late booking here, as the minutes tick down. Tottenham will be pleased with this though; a winning start to 2021 is just what they need."
3,78',Substitution,T. Ndombèlé,Lucas Moura,"Tottenham make their second change now, with Ndombele making way for Lucas Moura."
4,76',Substitution,H. Winks,M. Sissoko,Lloris gets a glance to this one and Ayling's return ball sails wide of the mark. Moussa Sissoko replaces Winks for the hosts.


# Entity Embedding for Categorical Variable

In [59]:
cont, cat = cont_cat_split(df, dep_var='commentary')
cont, cat

(['pr_time'], ['time', 'event', 'player_sub', 'main_player'])

In [16]:
from sklearn.model_selection import train_test_split

x_tr, x_te, y_tr, y_te = train_test_split(df.drop('commentary', axis=1), df['commentary'],
                                          test_size=0.33, random_state=42)

In [18]:
df_train = pd.concat([x_tr,y_tr], axis=1)
df_train

,time,event,player_sub,main_player,pr_time,commentary
13278,63',Substitution,A. Mac Allister,Y. Bissouma,63,"Meanwhile, Potter switches to a 3-5-2 as he replaces Mac Allister with an extra midfielder in Bissouma."
31005,44',Yellow Card,na,I. Diallo,44,Diallo pulls back Albrighton on the charge and goes into the book.
21149,42',Yellow Card,na,L. Ayling,42,"Ayling races across to challenge Trossard and he arrives a little late, catching the winger and earning himself a yellow card."
5720,82',Substitution,J. Bowen,A. Yarmolenko,82,"Bowen is replaced by Yarmolenko. Bowen has had a great game today, he's looked a real threat on the wing for the Hammers and looks absolutely shattered as he leaves the pitch."
30994,44',Red Card,na,A. Robinson,44,"FULHAM DOWN TO 10! Cavaleiro loses possession and Robinson tries to win it back. He flies into the challenge on Azpilicueta, completely missing the ball and taking out the right-back. The referee goes straight to his pocket and pulls out the red card."
...,...,...,...,...,...,...
6265,42',Yellow Card,na,L. Ayling,42,"Ayling races across to challenge Trossard and he arrives a little late, catching the winger and earning himself a yellow card."
11284,79',Substitution,Ivan Cavaleiro,J. Onomah,79,First change for Fulham now and it's Cavaleiro that's making way for Onomah.
38158,38',Goal,na,Fábio Silva,38,GOAL! FABIO SILVA LEVELS THINGS FOR WOLVES! 1-1! It is his first top-flight goal at Molineux! The corner sails past but is pumped back in and the striker plays a delightful one-two with Boly before lashing it past Button. All square in the Black Country derby!
860,44',Red Card,na,A. Robinson,44,"FULHAM DOWN TO 10! Cavaleiro loses possession and Robinson tries to win it back. He flies into the challenge on Azpilicueta, completely missing the ball and taking out the right-back. The referee goes straight to his pocket and pulls out the red card."


In [60]:
procs_nn = [Categorify]
device = torch.device('cpu')
to_nn = TabularPandas(df_train, procs_nn, cat,
                      y_names='commentary')
dls = to_nn.dataloaders(1024, device=device)

In [61]:
learn = tabular_learner(dls, layers=[1000, 700, 400, 300, 400], n_out=len(df['commentary']))
learn.fit_one_cycle(4, 5e-4)

epoch,train_loss,valid_loss,time
0,8.829665,None,00:16
1,5.487820,None,00:16
2,2.813770,None,00:16
3,1.569424,None,00:17


/home/ROHITGANGURDE/anaconda3/envs/a7/lib/python3.7/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [92]:
to_nn.xs

,time,event,player_sub,main_player
13278,43,5,4,151
31005,25,6,84,55
21149,23,6,84,87
5720,62,5,36,15
30994,25,4,84,12
...,...,...,...,...
6265,23,6,84,87
11284,58,5,35,69
38158,19,2,84,48
860,25,4,84,12


In [91]:
d = {'time': "41'", 'event': "Substitution", 'player_sub' : 'Thomas',
     'main_player':"Ziyech", 'pr_time':41}
ser = pd.Series(data=d, index=['time','event','player_sub','main_player','pr_time'])
preds, *_ = learn.predict(ser)
preds.show()

,time,event,player_sub,main_player,commentary
0,41',Substitution,#na#,#na#,Bryan is also brought on to replace Aina.


In [58]:
learn.save('learn8')

Path('models/learn8.pth')

In [78]:
def embed_features(learner, x):
    x = x.copy()
    for i, col in enumerate(learn.dls.cat_names):
        embed = learn.model.embeds[i]
        print(col)
        print(x[col])
        embed_data = embed(tensor(x[col], dtype=torch.int64).to(device))
        embed_names = [f'{col}_{j}' for j in range(embed_data.shape[1])]
        features = pd.DataFrame(data=embed_data, index=x.index, columns=embed_names)
        x = x.drop(col, axis=1)
        x = x.join(features)
    return x

In [68]:
embed = learn.model.embeds

In [87]:
embed

ModuleList(
  (0): Embedding(77, 18)
  (1): Embedding(7, 5)
  (2): Embedding(85, 19)
  (3): Embedding(156, 27)
)

In [88]:
df_train.iloc[0,2], embed[2](tensor(to.train.xs.iloc[0,2], dtype=torch.int64).to(device))

('A. Mac Allister',
 tensor([-0.0124,  0.0154, -0.0037, -0.0008, -0.0001,  0.0106, -0.0082,  0.0090,
          0.0018,  0.0104, -0.0039,  0.0066,  0.0021, -0.0082,  0.0035, -0.0036,
          0.0051, -0.0025,  0.0118], grad_fn=<EmbeddingBackward>))

In [64]:
procs = [Categorify]
to = TabularPandas(df_train, procs, cat, cont, 'commentary')

In [83]:
to.train.show()

,time,event,player_sub,main_player,pr_time,commentary
13278,63',Substitution,A. Mac Allister,Y. Bissouma,63,"Meanwhile, Potter switches to a 3-5-2 as he replaces Mac Allister with an extra midfielder in Bissouma."
31005,44',Yellow Card,na,I. Diallo,44,Diallo pulls back Albrighton on the charge and goes into the book.
21149,42',Yellow Card,na,L. Ayling,42,"Ayling races across to challenge Trossard and he arrives a little late, catching the winger and earning himself a yellow card."
5720,82',Substitution,J. Bowen,A. Yarmolenko,82,"Bowen is replaced by Yarmolenko. Bowen has had a great game today, he's looked a real threat on the wing for the Hammers and looks absolutely shattered as he leaves the pitch."
30994,44',Red Card,na,A. Robinson,44,"FULHAM DOWN TO 10! Cavaleiro loses possession and Robinson tries to win it back. He flies into the challenge on Azpilicueta, completely missing the ball and taking out the right-back. The referee goes straight to his pocket and pulls out the red card."
10373,72',Yellow Card,na,H. Ziyech,72,Ziyech is booked after he steps across Lookman to stop Fulham's counter-attack.
1289,90' + 5',Goal,na,H. Barnes,95,GOALLLLLLLLLLLLLLLLLLLLLLLL!!!!!! BARNES SEALS THE GAME!!! 2-0 LEICESTER!!! The home side break on the counter in the dying seconds and Tielemans puts Barnes through on goal. The winger holds his composure and slots his right-footed strike past a stranded McCarthy.
38707,38',Goal,na,Fábio Silva,38,GOAL! FABIO SILVA LEVELS THINGS FOR WOLVES! 1-1! It is his first top-flight goal at Molineux! The corner sails past but is pumped back in and the striker plays a delightful one-two with Boly before lashing it past Button. All square in the Black Country derby!
20984,78',Goal,na,M. Mount,78,"MOUNT SCORES! Chilwell's cross is palmed away from the goal by Areola, but he gives it straight to Mount, who is unmarked in the middle. He hits the shot on the volley and the keeper can't stop it from hitting the back of the net. 1-0 Chelsea!"
10328,37',Goal,na,J. Maddison,37,"WHAT A FINISH!!! MADDISON PUTS LEICESTER AHEAD FROM A TIGHT ANGLE!!! 1-0!! Out of nowhere the Foxes come alive and take the lead. Tielemans sends Maddison into space in the right inside channel and he displays a burst of pace to break away from Stephens. He drills a strike high towards the near post, blasting his attempt past McCarthy into the top-right corner. A super strike."


In [79]:
# learn = tabular_learner.load('learn8')
embed_xs = embed_features(learn, to.train.xs)

time
13278    43
31005    25
21149    23
5720     62
30994    25
         ..
6265     23
11284    58
38158    19
860      25
15795    63
Name: time, Length: 26416, dtype: int8
event
13278    5
31005    6
21149    6
5720     5
30994    4
        ..
6265     6
11284    5
38158    2
860      4
15795    5
Name: event, Length: 26416, dtype: int8
player_sub
13278     4
31005    84
21149    84
5720     36
30994    84
         ..
6265     84
11284    35
38158    84
860      84
15795    61
Name: player_sub, Length: 26416, dtype: int8
main_player
13278    151
31005     55
21149     87
5720      15
30994     12
        ... 
6265      87
11284     69
38158     48
860       12
15795     61
Name: main_player, Length: 26416, dtype: int16


In [67]:
embed_xs

,pr_time,time_0,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,...,main_player_17,main_player_18,main_player_19,main_player_20,main_player_21,main_player_22,main_player_23,main_player_24,main_player_25,main_player_26
13278,63,0.004884,0.002767,-0.005105,0.001646,0.019339,0.005746,0.007602,-0.015776,0.005001,...,0.001704,0.008490,0.013462,-0.009978,0.012518,0.008925,-0.006514,0.004712,0.002097,-0.005511
31005,44,0.018298,0.009842,-0.000175,0.003035,-0.001195,-0.009737,-0.009590,-0.007860,-0.021289,...,0.015694,-0.000088,0.007481,0.005568,0.008312,0.000473,0.003681,-0.004455,-0.016045,0.007111
21149,42,-0.001837,0.008236,0.004752,-0.011059,-0.000623,0.006003,0.004914,0.018300,-0.007308,...,-0.006986,0.006338,0.010310,0.000663,-0.005357,-0.004011,-0.010764,-0.006954,-0.001394,0.000513
5720,82,-0.005697,-0.003060,0.007421,0.004532,-0.002928,-0.005551,-0.004582,-0.007340,0.008844,...,-0.016851,-0.012104,-0.001068,-0.001435,-0.019272,-0.002514,0.001726,-0.009087,-0.007223,-0.009559
30994,44,0.018298,0.009842,-0.000175,0.003035,-0.001195,-0.009737,-0.009590,-0.007860,-0.021289,...,0.005457,-0.001575,-0.010771,0.001656,-0.000698,0.006656,0.005963,-0.005375,0.002567,0.015763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,42,-0.001837,0.008236,0.004752,-0.011059,-0.000623,0.006003,0.004914,0.018300,-0.007308,...,-0.006986,0.006338,0.010310,0.000663,-0.005357,-0.004011,-0.010764,-0.006954,-0.001394,0.000513
11284,79,-0.005639,0.005408,0.009322,0.000039,-0.010142,-0.019409,-0.006435,0.009085,-0.000924,...,0.000732,-0.011032,-0.004227,0.001948,0.003421,0.010307,-0.000239,0.004066,0.005708,0.009886
38158,38,-0.008849,-0.000216,0.000794,-0.003909,-0.001382,0.006802,0.012559,0.020757,-0.007469,...,0.005398,-0.005477,-0.005173,-0.011466,0.000172,-0.001235,-0.008024,-0.006572,-0.000222,0.011101
860,44,0.018298,0.009842,-0.000175,0.003035,-0.001195,-0.009737,-0.009590,-0.007860,-0.021289,...,0.005457,-0.001575,-0.010771,0.001656,-0.000698,0.006656,0.005963,-0.005375,0.002567,0.015763


In [350]:
to.ys

,commentary
0,2
1,132
2,140
3,241
4,157
...,...
39422,12
39423,60
39424,239
39425,251


In [277]:
embed_valid_xs = embed_features(learn, to.valid.xs)

In [278]:
embed_valid_xs

,time_0,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,time_9,...,main_player_20,main_player_21,main_player_22,main_player_23,main_player_24,main_player_25,main_player_26,main_player_27,main_player_28,main_player_29
26614,-0.021242,-0.006020,0.003531,0.011634,0.018741,0.007572,0.003504,0.009854,0.016033,0.011028,...,-0.002679,0.000861,0.012142,0.003357,-0.011981,0.012229,-0.002458,0.000768,-0.002417,-0.006615
28533,0.007693,0.004825,-0.010953,-0.013567,0.005793,0.012431,0.005530,0.007038,0.000105,0.004454,...,0.011084,-0.016017,0.001854,0.010596,-0.000100,0.001486,-0.024934,0.000604,-0.019131,0.006365
11568,0.012565,0.003069,-0.005915,-0.013634,0.007143,0.010780,0.009232,-0.003443,-0.002451,0.010082,...,-0.019058,-0.010733,0.017766,-0.003420,0.007331,-0.013142,-0.010556,-0.008791,0.005042,0.004618
7628,0.002883,0.005252,-0.008948,-0.005496,-0.008258,-0.002465,-0.002871,-0.014287,-0.008704,0.005342,...,-0.009660,0.001067,-0.011077,-0.015028,-0.000418,-0.001467,-0.000653,-0.005423,0.001942,0.008168
22395,0.018721,0.010925,-0.018227,-0.028583,-0.006423,0.010003,0.009473,-0.001753,0.016994,-0.000779,...,0.001289,0.006768,-0.000929,0.011695,0.021778,0.003233,-0.003960,-0.014444,-0.019404,0.031912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11142,0.014238,-0.006220,-0.002246,0.001548,-0.009250,0.006653,-0.007739,-0.008992,-0.003056,0.002443,...,0.002878,-0.017461,0.002731,-0.007421,0.015626,0.013654,-0.000473,-0.003548,-0.002914,0.007465
30134,-0.001226,0.002923,0.004506,-0.013412,-0.000674,0.001110,0.017062,0.000702,-0.010121,-0.022211,...,-0.014484,-0.001304,0.011523,-0.006265,-0.002075,-0.001508,-0.016273,-0.003331,0.002709,-0.016223
18378,0.008164,0.002036,-0.002127,0.003153,0.005036,-0.003247,0.007702,-0.006719,0.008519,0.012474,...,0.002599,0.021801,-0.012157,0.006728,-0.005496,0.007274,-0.019366,0.000078,-0.010061,0.009369
15645,0.013241,0.001374,0.001792,-0.017407,-0.005450,-0.002707,-0.024002,-0.012195,-0.007054,0.008149,...,0.005421,-0.001957,0.001166,-0.015784,-0.009983,0.000609,-0.011086,0.004046,-0.013029,-0.007875


In [291]:
# y_embed = embed_features(learn, to.train.ys)
ys = to.ys
show = to.show

In [318]:
to.train.ys.values.ravel()

array([167,  59,  85, ...,  84, 152,  30], dtype=int16)

In [346]:
row, clas, probs = learn.predict(df.iloc[10])
row.show()

,time,event,player_sub,main_player,commentary
0,50',Assist,,Son Heung-Min,A round of applause for that back-heel from Boly though. That's the assist of dreams! The dam finally breaks for West Brom. They will have to be on even greater guard now.


In [347]:
clas, probs

(tensor(3),
 tensor([9.0028e-05, 4.4254e-04, 3.4545e-05,  ..., 1.1167e-05, 4.7655e-05,
         1.4618e-05]))

In [68]:
save_pickle('data/emb_xs', embed_xs)

In [69]:
save_pickle('data/emb_valid_xs', embed_valid_xs)

In [10]:
embed_xs = load_pickle('emb_xs')

In [12]:
embed_valid_xs = load_pickle('emb_valid_xs')

# Multi-Layer Perceptron

In [230]:
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size,
                                   self.hidden_size)
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(p=0.5)
        self.fc2 = torch.nn.Linear(self.hidden_size, self.input_size)
        self.batch_norm = torch.nn.BatchNorm1d(self.input_size)
#         self.tanh2 = torch.nn.Tanh()
#         self.fc3 = torch.nn.Linear(self.input_size, 200, bias=True)
        
    def forward(self, x):
        fc1 = self.fc1(x)
        tanh = self.tanh(fc1)
        dropout = self.dropout(tanh)
        fc2 = self.fc2(dropout)
#         batch = self.batch_norm(fc2)
#         tanh = self.tanh2(batch)
#         output = self.fc3(tanh)
#         output = self.softmax(output)
        return fc2

In [15]:
x_train = embed_xs.iloc[:25000]
x_test = embed_xs.iloc[25000:]
# y_train = to.train.ys[:25000]
# y_test = to.train.ys[25000:2

In [16]:
x_train = torch.FloatTensor(x_train.values)
x_test = torch.FloatTensor(x_test.values)
# y_train = torch.FloatTensor(y_train.values)
# y_test = torch.FloatTensor(y_test.values)

In [231]:
model = Model(x_train.shape[1], 500)
criterion = torch.nn.KLDivLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [209]:
for epoch in range(3):
    
    optimizer.zero_grad()
    
    outputs = 

torch.Size([77, 25000])

In [232]:
model.eval()

Model(
  (fc1): Linear(in_features=77, out_features=500, bias=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=77, bias=True)
  (batch_norm): BatchNorm1d(77, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [233]:
model.eval()
h = model(x_train)
h.shape

torch.Size([25000, 77])

In [227]:
h

tensor([[ 0.4831,  0.4566, -0.0076,  ..., -0.1909, -0.3050,  0.3882],
        [ 0.5071,  0.3313, -0.0269,  ..., -0.3138, -0.3624,  0.3437],
        [ 0.4606,  0.4319, -0.0033,  ..., -0.1805, -0.2931,  0.3690],
        ...,
        [ 0.5133,  0.3685, -0.0353,  ..., -0.3045, -0.3499,  0.3497],
        [ 0.5357,  0.4605, -0.0427,  ..., -0.2724, -0.3356,  0.3835],
        [ 0.5357,  0.4605, -0.0427,  ..., -0.2724, -0.3356,  0.3835]],
       grad_fn=<NativeBatchNormBackward>)

In [234]:
U, b = model.fc2.weight, model.fc2.bias

In [235]:
U.shape

torch.Size([77, 500])

In [236]:
b.shape

torch.Size([77])

In [249]:
"""
        g_c = sigmoid(U_c*h + b_c)
    
    where  U_c is the weight parameter matrix
           b_c is the bias term
           h is the hidden representation
"""

def g(c):
    """
        c : index of the context word 
    """
    U_c = U[:,c].T
#     U_c = torch.reshape(U_c, (U_c.shape[0],1))
    b_c = b[c]
#     print(h.shape, U_c.shape, b_c.shape)
    g_c = torch.matmul(h, U_c) + b_c
#     g_c = g_c.detach().numpy()
    sigmoid = torch.nn.Sigmoid()
#     g_c = 1/(1 + np.exp(-g_c))
    g_c =  sigmoid(g_c)
    return g_c

In [246]:
# g(1).shape

g_c = torch.mm(h, U) #+ b
g_c.shape

torch.Size([25000, 500])

In [252]:
def loss():
    for c in range(x_train.shape[1]):
        g_c = g(c)
        for i in range(x_train.shape[0]):
            g_ck = g_c[i]
            word = x_train[i,c]
            print(word)
            break
        break

In [254]:
x_train[0,0]

tensor(17.)

In [256]:
x_train

tensor([[ 1.7000e+01, -6.9678e-03, -8.7373e-03,  ...,  1.5556e-03,
         -6.4716e-03, -2.5462e-03],
        [ 8.7000e+01, -8.9309e-03,  1.4219e-02,  ..., -1.3334e-02,
         -6.0982e-03, -8.5742e-03],
        [ 1.5000e+01,  1.5483e-03,  5.9269e-03,  ...,  6.8440e-03,
         -9.3233e-03, -3.3145e-04],
        ...,
        [ 7.2000e+01,  3.8889e-03,  1.8350e-02,  ..., -7.4292e-03,
         -7.6149e-04,  1.5664e-03],
        [ 4.4000e+01,  1.4752e-02, -6.6803e-03,  ...,  2.0876e-03,
          9.3981e-03, -7.5252e-03],
        [ 4.4000e+01,  1.4752e-02, -6.6803e-03,  ...,  2.0876e-03,
          9.3981e-03, -7.5252e-03]])